In [23]:
 # 머신러닝 도구 패키지 설치
!pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   -------- ------------------------------- 2.4/11.1 MB 12.3 MB/s eta 0:00:01
   ---------------- ----------------------- 4.7/11.1 MB 11.9 MB/s eta 0:00:01
   -------------------------- ------------- 7.3/11.1 MB 11.6 MB/s eta 0:00:01
   --------------------------------- ------ 9.4/11.1 MB 11.7 MB/s eta 0:00:01
   ---------------------------------------  11.0/11.1 MB 11.5 MB/s eta 0:00:01
   ---------------------------------------- 11.1/11.1 MB 10.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/41.2 MB ? eta -:--:--
   -- ------------------------------------- 2.1/41.2 MB 11.8 MB/s eta 0:00:04
   ---- ----------------------------------- 4.7/41.2 MB 11.9 MB/s eta 0:00:04
   ------ --------------------------------- 7.1/41.2 MB 11.8 MB/s eta 0:00:03
   -------- ------------------------------- 8.4/41.2 MB 11.6 MB/s eta 0:00:03


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from io import StringIO # 문자열을 파일처럼 사용할 수 있도록 지원하는 모듈

In [ ]:
# 데이터 준비 

csv_data = """A,B,C,D
1.0,2.0,3.0,4.0
5.0,6.0,,8.0
9.0,10.0,11.0,
"""

df = pd.read_csv(StringIO(csv_data))
df

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,9.0,10.0,11.0,NaN


In [12]:
# 결측 여부 확인
# df.info()
# df.isna()
# df.isnull() # isna()와 동일한 기능
# df.notna() # isna()와 반대 기능
# df.notnull() # notna()와 동일한 기능
df.isna().sum(axis=0) # False : 0, True : 1로 처리하기 때문에 sum()으로 결측값의 갯수 계산 가능

A    0
B    0
C    1
D    1
dtype: int64

In [21]:
# 결측 데이터 처리 1 : 행 / 열 삭제

df.dropna() # axis=0 # NaN이 있는 행 제거
df.dropna(axis=1) # NaN이 있는 열 제거
df.dropna(how='all') # 행의 모든 값이 NaN일 때 제거
df.dropna(how='any') # 행의 값 중 NaN이 한 개라도 있다면 제거
df.dropna(thresh=3) # 지정된 갯수 이상의 값이 NaN이 아닐 때 행 보존 아니면 제거
df.dropna(subset=['C']) # C컬럼의 값이 NaN인 행 제거

,A,B,C,D
0,1.0,2.0,3.0,4.0
2,9.0,10.0,11.0,NaN


In [27]:
# 결측 데이터 처리 2 : 다른 값으로 대체
from sklearn.impute import SimpleImputer # 결측 값 대체 기능 모듈

# imputer = SimpleImputer(strategy="mean") # median, most_frequence, constant
imputer = SimpleImputer(strategy="constant", fill_value=100) # constant인 경우 채울 값 지정
imputed_df = imputer.fit_transform(df)
imputed_df

array([[  1.,   2.,   3.,   4.],
       [  5.,   6., 100.,   8.],
       [  9.,  10.,  11., 100.]])

In [28]:
# 데이터 준비
df2 = pd.DataFrame([
    ['green', 'M', 10.1, 'class1'],
    ['red', 'L', 13.5, 'class2'],
    ['blue', 'XL', 15.3, 'class1'],
])

df2.columns = ['color', 'size', 'price', 'classlabel']
df2

,color,size,price,classlabel
0,green,M,10.1,class1
1,red,L,13.5,class2
2,blue,XL,15.3,class1


In [40]:
# 범주 문자열 -> 숫자로 변환

df2['classlabel'].unique() # 중복 제거하고 고유한 값만 반환
df2['classlabel'].value_counts() # 중복 제거하고 고유한 값만 갯수와 함께 반환
np.unique(df2['classlabel'], return_counts=True) # 중복 제거하고 고유한 값만 반환 ( 갯수도 반환 가능 )

# mapping = {'class1':1, 'class2':2} # 변환 규칙 dict로 정의
mapping = {
    v: idx for idx, v in enumerate( df2['classlabel'].unique() )
}

df2['classlabel'] = df2['classlabel'].map(mapping) # 변환 규칙을 처리해서 값 변환 처리
df2

,color,size,price,classlabel
0,green,M,10.1,0
1,red,L,13.5,1
2,blue,XL,15.3,0


In [42]:
# 수치화된 범주형 데이터를 원래 데이터로 복원
inverse_mapping = {
    v:k for k, v in mapping.items()
}
inverse_mapping

df2['classlabel'] = df2['classlabel'].map(inverse_mapping)
df2


,color,size,price,classlabel
0,green,M,10.1,class1
1,red,L,13.5,class2
2,blue,XL,15.3,class1


In [ ]:
# 범주형 데이터를 수치형 데이터로 변환 2 : LabelEncoder 사용

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
transformed = le.fit_transform(df2['classlabel']) # 범주형 데이터 -> 숫자 데이터 변환
transformed
df2['classlabel'] = transformed
df2

,color,size,price,classlabel
0,green,M,10.1,0
1,red,L,13.5,1
2,blue,XL,15.3,0


In [46]:
# 수치형 데이터를 범주형 데이터로 복원 2 : LabelEncoder 사용

df2['classlabel'] = le.inverse_transform(transformed)
df2

,color,size,price,classlabel
0,green,M,10.1,class1
1,red,L,13.5,class2
2,blue,XL,15.3,class1


In [51]:
df2['color'].values
df2[['color']].values

array([['green'],
       ['red'],
       ['blue']], dtype=object)

In [54]:
# 범주형 데이터 수치형 데이터로 변환 3 : OneHotEncoder 사용

from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()
transformed = ohe.fit_transform(df2[['color']])
print( transformed )
transformed.toarray() # sparse matrix expression -> dense matrix expression


<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 3 stored elements and shape (3, 3)>
  Coords	Values
  (0, 1)	1.0
  (1, 2)	1.0
  (2, 0)	1.0


array([[0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.]])

In [57]:
pd.get_dummies( df2['color'] ) # OneHotEncoding 처리
pd.get_dummies( df2['color'] ).astype(int) # OneHotEncoding 처리
pd.get_dummies( df2['color'], drop_first=True ) # OneHotEncoding 처리

,green,red
0,True,False
1,False,True
2,False,False


In [59]:
df_wine = pd.read_csv('data-files/winequality-red.csv', sep=";")
df_wine.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [66]:
df_wine.min(axis=0).values, df_wine.max(axis=0).values

(array([4.6    , 0.12   , 0.     , 0.9    , 0.012  , 1.     , 6.     ,
        0.99007, 2.74   , 0.33   , 8.4    , 3.     ]),
 array([ 15.9    ,   1.58   ,   1.     ,  15.5    ,   0.611  ,  72.     ,
        289.     ,   1.00369,   4.01   ,   2.     ,  14.9    ,   8.     ]))

In [ ]:
from sklearn.preprocessing import MinMaxScaler

mms = MinMaxScaler()
scaled_wine = mms.fit_transform(df_wine)
scaled_wine.min(axis=0), scaled_wine.max(axis=0)

(array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]))